In [22]:
# import jax.numpy as np
import numpy as np
import gym

import tensorflow as tf
tf.enable_eager_execution()

In [23]:
env = gym.make('LunarLander-v2')

/home/telfaralex/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [24]:
def play_episode(player, env, len_episode=100):
    obs = env.reset()
    trajectory = []
    old_obs = obs
    done = False
    
    # need fixed length trajectories
    for i in range(len_episode):
        if not done:
            a = player(obs)
            obs, r, done, info = env.step(a)
            
        else:  # pad with zeros
            a = 0
            obs, r, done, info = (np.zeros(old_obs.shape), 0, done, None)

        trajectory.append(np.concatenate([np.array([a]), old_obs, np.array([r])]))
        old_obs = obs
    return np.vstack(trajectory)

In [25]:
rnd_player = lambda s: env.action_space.sample()

data = play_episode(rnd_player, env, 101)
actions = data[:-1, 0]
deltas = data[1:, 1:] - data[:-1, 1:]

In [26]:
deltas.shape, actions.shape

((100, 9), (100,))

## Independent

Want a set of actions that are orthogonal / modular. (disentangled)

$$
\begin{align}
\Delta s(a) = \mathbb E[\tau(s, a) - s] \\
\\
\forall i\neq j:i,j \in [1:K] \;\; \\
\langle\Delta s(f(w_i)), \Delta s(f(w_j)) \rangle = 0 \\
\end{align}
$$


In [36]:
expected_deltas = tf.math.unsorted_segment_mean(tf.constant(deltas), tf.constant(actions, tf.int32), 4)
expected_deltas

<tf.Tensor: id=48, shape=(4, 9), dtype=float64, numpy=
array([[-1.48288241e-02, -4.24741790e-03, -4.04429232e-02,
         2.77148169e-02,  3.13965139e-02,  1.30061496e-02,
        -2.43902439e-02,  0.00000000e+00,  2.09429161e+00],
       [ 1.03362994e-02, -2.21525815e-02, -5.82436100e-03,
        -2.21881354e-02, -2.12912476e-02,  4.57255105e-02,
         0.00000000e+00,  0.00000000e+00, -2.19994062e+00],
       [ 1.09077700e-02, -2.27179063e-02,  4.95382798e-02,
         3.90037424e-03, -2.22582868e-02, -2.92850011e-03,
         0.00000000e+00,  0.00000000e+00,  2.09208150e+00],
       [ 9.53963174e-03, -1.86591316e-02,  6.73618664e-03,
        -3.03167682e-02, -2.15115810e-02, -4.42639279e-02,
         4.16666667e-02,  0.00000000e+00, -3.67450803e+00]])>

## Regular

Learn a consistent action space
Want to learn a space where the actions act regularly.

$$
\begin{align}
\Delta s(a) = \mathbb E[\tau(s, a) - s] \\
\\
\exists c_i\in \mathbb R^d \;\;\text{s.t.} \\
\parallel \Delta s(a_i) - c_i \parallel_2^2 =0\\
\end{align}
$$

Should point in the same direction, and have the same magnitude.
Does it really need to have the same magnitude??